google job search scraping tuto : https://serpapi.com/blog/scrape-google-jobs-organic-results-with-python/


## IMPORT NECESSARY LIBRARIES

In [1]:
import pandas as pd
import csv 

# Libraries for using google jobs API from SERPAPI
from serpapi import GoogleSearch

# library for using SERPSTACK
#import requests

#from google.cloud import bigquery

import json, os
import datetime

# Library to generate UULE code on the fly
import uule_grabber

## GET GEOTARGETS INTO UULE CODE

In [2]:
# assign all google geo location to dataset
geotargets = pd.read_csv('geotargets-2022-12-21.csv') 
# get canonical name for our cities of interest >>> PARIS for instance

canonical_name = geotargets.loc[(geotargets['Target Type'] == 'Country') &( geotargets['Country Code'] == 'FR'), ['Canonical Name']]

#canonical_name = geotargets.loc[(geotargets.Name == 'Nantes') & (geotargets['Country Code'] == 'FR') & (geotargets['Target Type'] == 'City'), ['Canonical Name']]
#canonical_name = canonical_name.iloc[0,0]
canonical_name = canonical_name.iloc[0,0]

# get corresponding uule code to cities of interest
cities = canonical_name
uule_codes = uule_grabber.uule(cities)

## COLLECT DATA W\ SERPAPI

In [3]:
# Defining our search query + necessary parameters for GoogleSearch object

search_queries = ["machine learning engineer", "data scientist", "data analyst", "data engineer"]

# initialize jobs_all outside of the loop
jobs_all = pd.DataFrame()
#for date in datelist:
for query in search_queries:

    # serpapi will iterate up to n number of iterations
    for num  in range(45):

        start = num * 10


    # define parameters
        params = {
            'api_key': '4b799b64af09be918f6d66d6e908184cba836c46596e58bfa8bf1fb9280e7f09',                              
            'device':'desktop',
            'uule': uule_codes,                         # encoded location 
            'q': query,                          # search query
            'google_domain': 'google.fr',              
            'hl': 'fr',                                 # language of the search
            'gl': 'fr',                                 # country of the search
            'engine': 'google_jobs',                    # SerpApi search engine
            'start': start,                             # pagination
            'chips': 'date_posted:today'  #'data_range:2023-02-03'   #'date_posted:today'                 
        }

        # get results 
        search = GoogleSearch(params)
        results = search.get_dict()  # JSON file to python dict 

        # check if last search page, exceptions handling   
        try:
            if results['error'] == "Google hasn't returned any results for this query.":
                    break
        except KeyError:
                print(f"Getting SerpAPI data for page: {start} of '{query}' results")
        else:
                continue

        # create dataframe of 10 pulled results
        jobs = results['jobs_results']
        jobs = pd.DataFrame(jobs)
        jobs = pd.concat([pd.DataFrame(jobs), 
                        pd.json_normalize(jobs['detected_extensions'])], #convert detected extension key in json files into pandas df
                        axis=1).drop('detected_extensions', 1) # drop json object
        jobs['date_time'] = datetime.datetime.now() # add extraction date column for job results

        # concat dataframe
        if start == 0:
                jobs_all = jobs
        else:
                jobs_all = pd.concat([jobs_all, jobs])


        jobs_all['search_query'] = query


    # exporting results to csv file
    jobs_all.to_csv('../data/gg_job_search_all_RAW.csv', mode='a', header=False, index=True)


https://serpapi.com/search
Getting SerpAPI data for page: 0 of 'machine learning engineer' results
https://serpapi.com/search


/tmp/ipykernel_66754/3824445398.py:46: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  jobs = pd.concat([pd.DataFrame(jobs),


Getting SerpAPI data for page: 10 of 'machine learning engineer' results
https://serpapi.com/search
Getting SerpAPI data for page: 20 of 'machine learning engineer' results
https://serpapi.com/search
Getting SerpAPI data for page: 30 of 'machine learning engineer' results
https://serpapi.com/search
https://serpapi.com/search
Getting SerpAPI data for page: 0 of 'data scientist' results
https://serpapi.com/search
Getting SerpAPI data for page: 10 of 'data scientist' results
https://serpapi.com/search
Getting SerpAPI data for page: 20 of 'data scientist' results
https://serpapi.com/search
Getting SerpAPI data for page: 30 of 'data scientist' results
https://serpapi.com/search
Getting SerpAPI data for page: 40 of 'data scientist' results
https://serpapi.com/search
Getting SerpAPI data for page: 50 of 'data scientist' results
https://serpapi.com/search
Getting SerpAPI data for page: 60 of 'data scientist' results
https://serpapi.com/search
Getting SerpAPI data for page: 70 of 'data scientis

In [9]:
import datetime

datetime.datetime.now

!git add gg_job_search_scrape.ipynb
!git commit -m "daily scrape"
!git push

On branch master
Your branch is up to date with 'origin/master'.

Changes not staged for commit:
  (use "git add/rm <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   ../data/gg_job_search_all_RAW.csv
	deleted:    ../notebooks/notebooks_models/.ipynb_checkpoints/Untitled-checkpoint.ipynb
	deleted:    ../notebooks/notebooks_models/.ipynb_checkpoints/Untitled1-checkpoint.ipynb
	deleted:    ../notebooks/notebooks_models/Untitled.ipynb
	deleted:    ../notebooks/notebooks_models/Untitled1.ipynb

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	../.mrx-link-logs/mrx-link-27619.log
	../notebooks/notebooks_models/.mrx-link-logs/Untitled1.ipynb.log

no changes added to commit (use "git add" and/or "git commit -a")
Everything up-to-date
